In [1]:
using JuMP, CSV, DataFrames, Gurobi, Interpolations

In [3]:
# Read the test and train CSV files
data_targeted = CSV.File("Data_Targeted.csv") |> DataFrame;
data_email = CSV.File("Data_Emails.csv") |> DataFrame;
data_print = CSV.File("Data_Print.csv") |> DataFrame;
data_influencer = CSV.File("Data_Influencers.csv") |> DataFrame;

In [4]:
B = [400.0, 800.0, 1000.0, 1200.0, 1500.0, 2000.0, 2500.0, 4000.0, 5000.0];

In [5]:
# Creating linear interpolation functions for each marketing option
interp_targeted = LinearInterpolation(data_targeted[!, :Cost], data_targeted[!, :Views]);
interp_email = LinearInterpolation(data_email[!, :Cost], data_email[!, :Views]);
interp_print = LinearInterpolation(data_print[!, :Cost], data_print[!, :Views]);
interp_influencers = LinearInterpolation(data_influencer[!, :Cost], data_influencer[!, :Views]);

## BASELINE 

In summary, linear interpolation is used here to create a continuous function from your discrete data points, allowing the optimization model to work effectively with any investment amount within the provided ranges. It bridges the gap between your data and the needs of the optimization solver.

In [6]:
for Budget in B
        # Optimization model
        model = Model(Gurobi.Optimizer)
        
        # Decision variables for investments
        @variable(model, investment_targeted >= 0)
        @variable(model, investment_email >= 0)
        @variable(model, investment_print >= 0)
        @variable(model, investment_influencers >= 0)
        
        # Additional variables and constraints for piecewise linear segments for each marketing option
        # Helps in modeling non-linear relationships
        
        # Creating a piecewise linear function
        # Targeted Online Advertising
        @variable(model, targeted_segments[1:3] >= 0) #each col is non-negative
        
        breakpoints_targeted = [0, 40, 310, 990] 
        # Example breakpoints 
        #defines the investment levels at which the slope (rate of increase in views) changes. 
        #These breakpoints divide the investment range into segments.
        
        slopes_targeted = [3, 4, 5] 
        # Example slopes
        # represents the rate of change in views per unit of investment in each segment. For instance, a slope of 0.8 for the 
        #first segment means that for each unit of investment in this segment, the views increase by 0.8.
        
        @constraint(model, investment_targeted == sum(targeted_segments)) #the sum of investments across all segments
        
        @constraint(model, sum(targeted_segments[i] for i in 1:3) <= breakpoints_targeted[end])
        for i in 1:3
            @constraint(model, targeted_segments[i] <= breakpoints_targeted[i+1] - breakpoints_targeted[i])
        end
        
        # Email Marketing - done
        # Example breakpoints and slopes for Email Marketing
        @variable(model, email_segments[1:3] >= 0) 
        breakpoints_email = [0, 77.05, 144.87, 300] # Example breakpoints 
        slopes_email = [18, 19.99, 19.95] # Example slopes
        @constraint(model, investment_email == sum(email_segments))
        @constraint(model, sum(email_segments[i] for i in 1:3) <= breakpoints_email[end])
        for i in 1:3
            @constraint(model, email_segments[i] <= breakpoints_email[i+1] - breakpoints_email[i])
        end
        
        # Print Media - done
        # Example breakpoints and slopes for Print Media
        @variable(model, print_segments[1:3] >= 0)
        slopes_print = [12, 40, 73] # Example breakpoints
        breakpoints_print = [200, 375, 565, 695] # Example slopes
        @constraint(model, investment_print == sum(print_segments))
        @constraint(model, sum(print_segments[i] for i in 1:3) <= breakpoints_print[end])
        for i in 1:3
            @constraint(model, print_segments[i] <= breakpoints_print[i+1] - breakpoints_print[i])
        end
        
        # Influencer Marketing - done
        # Example breakpoints and slopes for Influencer Marketing
        @variable(model, influencers_segments[1:3] >= 0)
        breakpoints_influencers = [100, 298, 406, 694] # Example breakpoints
        slopes_influencers = [10, 6, 2] # Example slopes
        @constraint(model, investment_influencers == sum(influencers_segments))
        @constraint(model, sum(influencers_segments[i] for i in 1:3) <= breakpoints_influencers[end])
        for i in 1:3
            @constraint(model, influencers_segments[i] <= breakpoints_influencers[i+1] - breakpoints_influencers[i])
        end
        
        # Objective: Maximize views
        @objective(model, Max, sum(slopes_targeted[i] * targeted_segments[i] for i in eachindex(slopes_targeted)) + 
                      sum(slopes_email[i] * email_segments[i] for i in eachindex(slopes_email)) +
                      sum(slopes_print[i] * print_segments[i] for i in eachindex(slopes_print)) +
                      sum(slopes_influencers[i] * influencers_segments[i] for i in eachindex(slopes_influencers)))

        
        # Budget constraint
            @constraint(model, investment_targeted + investment_email +
                           investment_print + investment_influencers <= Budget)

        # Baseline constraints
        @constraint(model, investment_targeted <= Budget/4)
        @constraint(model, investment_email <= Budget/4)
        @constraint(model, investment_print <= Budget/4)
        @constraint(model, investment_influencers <= Budget/4)
        
        # Solve the model
        optimize!(model)
        
        # Results
        println("Optimal Investments with Budget = ", Budget)
        println("Targeted Online Advertising: \$", value(investment_targeted))
        println("Email Marketing: \$", value(investment_email))
        println("Print Media: \$", value(investment_print))
        println("Influencer Marketing: \$", value(investment_influencers))
        println("Total Views: ", objective_value(model))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-10
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 25 rows, 16 columns and 48 nonzeros
Model fingerprint: 0xae7f2303
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+03]
Presolve removed 25 rows and 16 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.0797713e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  1.079771280e+04

User-callback calls 45, time in user-callback 0.00 sec
Optimal Investments with Budget = 400.0
Targeted Online Advertising: $100.0
Email Marketing: $100.0
Print Media: $100.0
Inf

## MIO 

In [8]:
BigM = 10000

10000

In [15]:
for Budget in B
        # Optimization model
        model = Model(Gurobi.Optimizer)
        
       # Decision variables for investments
        @variable(model, investment_targeted >= 0)
        @variable(model, investment_email >= 0)
        @variable(model, investment_print >= 0)
        @variable(model, investment_influencers >= 0)
        
        # Binary variables for onboarding fee
        @variable(model, onboard_targeted, Bin)
        @variable(model, onboard_email, Bin)
        @variable(model, onboard_print, Bin)
        @variable(model, onboard_influencers, Bin)
        
        # Onboarding fee constraints
        @constraint(model, investment_targeted <= onboard_targeted * BigM)
        @constraint(model, investment_email <= onboard_email * BigM)
        @constraint(model, investment_print <= onboard_print * BigM)
        @constraint(model, investment_influencers <= onboard_influencers * BigM)

        # Additional variables and constraints for piecewise linear segments for each marketing option
        # Helps in modeling non-linear relationships
        
        # Creating a piecewise linear function
        # Targeted Online Advertising
        @variable(model, targeted_segments[1:3] >= 0) #each col is non-negative
        
        breakpoints_targeted = [0, 40, 310, 990] 
        # Example breakpoints 
        #defines the investment levels at which the slope (rate of increase in views) changes. 
        #These breakpoints divide the investment range into segments.
        
        slopes_targeted = [3, 4, 5] 
        # Example slopes
        # represents the rate of change in views per unit of investment in each segment. For instance, a slope of 0.8 for the 
        #first segment means that for each unit of investment in this segment, the views increase by 0.8.
        
        @constraint(model, investment_targeted == sum(targeted_segments)) #the sum of investments across all segments
        
        @constraint(model, sum(targeted_segments[i] for i in 1:3) <= breakpoints_targeted[end])
        for i in 1:3
            @constraint(model, targeted_segments[i] <= breakpoints_targeted[i+1] - breakpoints_targeted[i])
        end
        
        # Email Marketing - done
        # Example breakpoints and slopes for Email Marketing
        @variable(model, email_segments[1:3] >= 0) 
        breakpoints_email = [0, 77.05, 144.87, 300] # Example breakpoints 
        slopes_email = [18, 19.99, 19.95] # Example slopes
        @constraint(model, investment_email == sum(email_segments))
        @constraint(model, sum(email_segments[i] for i in 1:3) <= breakpoints_email[end])
        for i in 1:3
            @constraint(model, email_segments[i] <= breakpoints_email[i+1] - breakpoints_email[i])
        end
        
        # Print Media - done
        # Example breakpoints and slopes for Print Media
        @variable(model, print_segments[1:3] >= 0)
        slopes_print = [12, 40, 73] # Example breakpoints
        breakpoints_print = [200, 375, 565, 695] # Example slopes
        @constraint(model, investment_print == sum(print_segments))
        @constraint(model, sum(print_segments[i] for i in 1:3) <= breakpoints_print[end])
        for i in 1:3
            @constraint(model, print_segments[i] <= breakpoints_print[i+1] - breakpoints_print[i])
        end
        
        # Influencer Marketing - done
        # Example breakpoints and slopes for Influencer Marketing
        @variable(model, influencers_segments[1:3] >= 0)
        breakpoints_influencers = [100, 298, 406, 694] # Example breakpoints
        slopes_influencers = [10, 6, 2] # Example slopes
        @constraint(model, investment_influencers == sum(influencers_segments))
        @constraint(model, sum(influencers_segments[i] for i in 1:3) <= breakpoints_influencers[end])
        for i in 1:3
            @constraint(model, influencers_segments[i] <= breakpoints_influencers[i+1] - breakpoints_influencers[i])
        end
        
        # Objective: Maximize views
        @objective(model, Max, sum(slopes_targeted[i] * targeted_segments[i] for i in eachindex(slopes_targeted)) + 
                      sum(slopes_email[i] * email_segments[i] for i in eachindex(slopes_email)) +
                      sum(slopes_print[i] * print_segments[i] for i in eachindex(slopes_print)) +
                      sum(slopes_influencers[i] * influencers_segments[i] for i in eachindex(slopes_influencers)))
    
        # Budget constraint
            @constraint(model, investment_targeted + investment_email +
                           investment_print + investment_influencers + 
            200*(onboard_targeted + onboard_email + onboard_print + onboard_influencers) <= Budget)
        # Solve the model
        optimize!(model)
        
        # Results
        println("Optimal Investments with Budget = ", Budget)
        println("Targeted Online Advertising: \$", value(investment_targeted))
        println("Email Marketing: \$", value(investment_email))
        println("Print Media: \$", value(investment_print))
        println("Influencer Marketing: \$", value(investment_influencers))
        println("Total Views: ", objective_value(model))

        # Check which investments have been made and calculate onboarding fees
        onboarding_fees_paid = 0
        total_investment = 0
        
        # Create a dictionary to map the investment decision variables to their names and binary onboarding variables
        investments = Dict(
            "Targeted Online Advertising" => (investment_targeted, onboard_targeted),
            "Email Marketing" => (investment_email, onboard_email),
            "Print Media" => (investment_print, onboard_print),
            "Influencer Marketing" => (investment_influencers, onboard_influencers)
        )
        
        println("Investments with onboarding fees:")
        for (name, (invest_var, onboard_var)) in investments
            invest_value = value(invest_var)
            onboard_value = value(onboard_var)
            total_investment += invest_value
            if onboard_value == 1
                onboarding_fees_paid += 200
                println("$name: \$", invest_value, " (Onboarding Fee Paid)")
            else
                println("$name: \$", invest_value)
            end
        end
        
        println("Total Onboarding Fees Paid: \$", onboarding_fees_paid)
        println("Total Investment Amount: \$", total_investment)
        println("Total Views: ", objective_value(model))
end

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-10
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 25 rows, 20 columns and 56 nonzeros
Model fingerprint: 0x7df6bd8f
Variable types: 16 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [2e+00, 7e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+03]
Found heuristic solution: objective -0.0000000
Presolve removed 20 rows and 6 columns
Presolve time: 0.00s
Presolved: 5 rows, 14 columns, 28 nonzeros
Variable types: 10 continuous, 4 integer (4 binary)

Root relaxation: objective 1.495667e+04, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0  